In [59]:
import skweak
import gensim
import spacy
import re
import pandas as pd
import pickle as pk
import itertools
import os

from skweak.base import CombinedAnnotator
from spacy.training import iob_to_biluo
from typing import List, Set
from tqdm.auto import tqdm
from tqdm import tqdm


from typing import List


tqdm.pandas()

In [ ]:
nlp = spacy.load('nl_core_news_lg', disable = ['lemmatizer','ner','attribute_ruler','tagger'])

In [3]:
test_dataset = "dataset/only_known_products_stratified_sample.csv.gz"

In [4]:
useful_columns = ['my_problem_is']
os.getcwd()
df = pd.read_csv(test_dataset, delimiter=",",usecols=useful_columns)

In [5]:
DIEPROFUN = ["pakket","hussel","kpn een mkb", "kpn 1 mkb", "kpn_een", "kpn een", 
               "contract",'bundel','dienst','databundel',"data", "sim only","mijnkpn"
               ,'cloud','dsl',"wifi","glasvezel", "smartphone","mobiel","mobiele",
               "mobieltje", "uploadsnelheid","downloadsnelheid","snelheid", "gigabyte", 
               'iphone','samsung','laptop', "viaplay", "netflix", "spotify", "eherkenning"
               , "simkaart", "simkaartje","simkaartnummer","pukcode","pin", "pinautomaat",
               "pinnummer","puk","site",'portal','mijnkpn','kpn',
              ]
NETWORK = ["internet","4g","5g","draad",  "signaal", "punt", "lijn","kanaal","kabel", "draad", "dns", "bereik", "bandbreedte", "ader"]
# ADDRESS = ['adres']
HARDWARE = ["modem","router","fritzbox","fritz","experia","experiabox","tv", "televisie", "television", "pc","computer"
            "versterker","toestel","camera","card","switch","lamp","kast", "zekeringskast","pas", "meterkast", "desktop", "laptop" , "camera",
           "apparaat", ]

TIJD = ['dag','week','uur','maand','maandag','tijd',"vandaag","nacht"]
COMPETITOR = ['ziggo','vodafone']
PROBLEM = ["fout","fouten","error","probleem","problemen","storing","incident", "schade", "klacht", "conflict"
                  "firewall","virus","virussen",'fraude','oplichting','melding']
PERSON = ["ik", "we","Ik","We"]
BILLING = ['factuur','facturen','incassobureau','termijnen','afbetaling'
            ,"euro",'kosten','geld','betaling',"korting", 'betalingsachterstand', "bankrekening"]

COMMUNICATION = ['sms','mail','voicemail']
CONTRACT = ['abonnement', 'contract','abonnementje']
LOCATION = ['locatie','huis','kantoor','winkel','stationlocatie','home', "etage", "adress", "wijk", "buurt"]

MECHANIC = ['monteur']
PROCESS = ['work', 'vertraging', 'verplaatsing','verhuizing','transitie','renovatie','proces','implementatie','actie']
CONNECTION = ['verbinding', 'aansluiting','koppeling']
NUMBER = ['nummer','telefoonnummer']
ORDER = ['order', 'opdracht', 'bestelling']
OFFER = ["aanvraag","aanbieding", "offerte"]
PLANNING = ["week", "bevestiging", "afspraak"]
SEGMENT = ["segment", "kleinzakelijk", "grootzakelijk"]
OTHER = ['cadeau', "verpakking", "functie", "test", "oorzaak",'status','blokkering']

In [6]:
original_ner_classes = [DIEPROFUN,NETWORK,HARDWARE,TIJD,COMPETITOR,PROBLEM,PERSON,BILLING,COMMUNICATION,CONTRACT,LOCATION, MECHANIC, PROCESS,
                        CONNECTION, NUMBER, ORDER, OFFER, PLANNING, SEGMENT, OTHER]

In [7]:
similarity_model = gensim.models.KeyedVectors.load("MODEL_GOES_HERE")

In [8]:
def add_candidate_synonyms(ner_class: List[str]):
    candidates = []
    print(ner_class)
    for word in ner_class:
        try:
            top10_most_similar = similarity_model.wv.most_similar(word, topn=30)
            top10_most_similar = [word[0].replace("_"," ") for word in top10_most_similar if word[1] >= 0.80 and not (word[0] in ['haha','heb','oke'])]
            top10_most_similar = [word for word in top10_most_similar if len(word.split(" ")) < 2]
            candidates.append(top10_most_similar)
        except:
            print("{} is OOV".format(word))
            continue
    return candidates

In [ ]:
updated_labels = []
for ner_class in original_ner_classes:
    candidates = add_candidate_synonyms(ner_class)
    candidates = candidates
    merged = list(itertools.chain(*candidates)) + ner_class

    updated_labels.append(list(set(merged)))

assert len(original_ner_classes) == len(updated_labels)


In [11]:
transp = pd.DataFrame(data=updated_labels,index=None).transpose()

In [12]:
transp.columns = labels

NameError: name 'labels' is not defined

In [143]:
transp.to_csv("expanded_ner_classes.csv", encoding='utf8')

In [13]:
for index,ner_class in enumerate(updated_labels):
    print(index, ner_class)

0 ['simkaarten', 'viaplay', 'pin', 'gigabyte', 'ongelimiteerd', 'tumtum', 'giechelen', 'glasvezels', 'laptop', 'spotify', 'uitzendt', 'uitgediend', 'tewerkstellingsvergunning', 'pinautomaat', 'computer', 'erkenning', 'kassasysteem', 'wifiverbinding', 'mobiele', 'hussel', 'snelheid', 'lokmiddel', 'bundel', 'ongelimiteerde', 'android', 'tablet', 'gigabytes', 'simcard', 'alarmsysteem', 'internetsnelheden', 'kpn 1 mkb', 'portal', 'porto', 'mbits', 'onbeperkt', 'internetsnelheid', 'pakketten', 'pincode', 'glasvezelaansluiting', 'contract', 'mobieltje', 'bundeltje', 'iphone', 'site', 'snelheden', 'mijnkpn', 'russell', 'smartphone', 'pinnummer', 'unlimited', 'uploadsnelheid', 'dsl', 'gb', 'glasvezelverbinding', 'website', 'gig', 'data', 'netflix', 'databundel', 'giechel', 'simkaartje', 'internetbundel', 'internetpakket', 'simkaart', 'glasvezel', 'sim', 'aanleggen', 'pukcode', 'koper', 'kpn een mkb', 'glasvezelkabel', 'contracten', 'dienst', 'kpn_een', 'portaal', 'kpn een', 'voordeelbundel', '

In [14]:
def DIEPROFUN_entities_detector(doc):

    search_range = 2
    for index, tok in enumerate(nlp(doc.text)):
      if tok.text in updated_labels[0]:
        start = tok.i
        if index > 0:
          distance_from_start = index - 0
          # print(distance_from_start)
          if distance_from_start < 2:
            search_range = distance_from_start

          reverse = -1
          while ((tok.nbor(reverse).pos_ in ['PROPN', 'NOUN', 'ADJ', 'SYM'] or ["een","1"]) and -abs(search_range) < reverse):
            if tok.nbor(reverse).pos_ in ['PUNCT', 'VERB', 'DET', 'ADP','PRON']:
              break
            start = tok.nbor(reverse).i
            reverse -= 1
        
        yield start, index+1, "DIEPROFUN"

def NETWORK_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[1]:
            yield index, index+1, "NETWORK"
            

def HARDWARE_entities_detector(doc):

    search_range = 2
    for index, tok in enumerate(nlp(doc.text)):
      if tok.text in updated_labels[2]:
        start = tok.i
        if index > 0:
          distance_from_start = index - 0
          # print(distance_from_start)
          if distance_from_start < 2:
            search_range = distance_from_start

          reverse = -1
          while ((tok.nbor(reverse).pos_ in ['PROPN', 'NOUN', 'ADJ', 'SYM'] or ["een","1"]) and -abs(search_range) < reverse):
            if tok.nbor(reverse).pos_ in ['PUNCT', 'VERB', 'DET', 'ADP','PRON']:
              break
            start = tok.nbor(reverse).i
            reverse -= 1
        
        yield start, index+1, "HARDWARE"
        

def TIME_entities_detector(doc):
    search_range = 2
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text in updated_labels[3]:
            start = tok.i
            if index > 0:
              distance_from_start = index - 0
              if distance_from_start < 2:
                search_range = distance_from_start
              reverse = -1
              while ((tok.nbor(reverse).pos_ in ['NUM']) and -abs(search_range) < reverse):
                  if tok.nbor(reverse).pos_ in ['PUNCT', 'VERB', 'DET', 'ADP', 'POSS', 'PRON']:
                    break
                  start = tok.nbor(reverse).i
                  reverse -= 1

            yield start, index+1, "TIME"
            
            
def COMPETITOR_entities_detector(doc):
     for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[4]:
            yield index, index+1, "COMPETITOR" 

def PROBLEM_entities_detector(doc):
     for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[5]:
            yield index, index+1, "PROBLEM"

def PERSON_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[6]:
            yield index, index+1, "PERSON" 
            
def BILLING_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[7]:
            yield index, index+1, "BILLING"
            
def COMMUNICATION_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[8]:
            yield index, index+1, "COMMUNICATION" 

def CONTRACT_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[9]:
            yield index, index+1, "CONTRACT"

def LOCATION_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[10]:
            yield index, index+1, "LOCATION"
            
def MECHANIC_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[11]:
            yield index, index+1, "MECHANIC"
            
def PROCESS_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[12]:
            yield index, index+1, "PROCESS"
            
def CONNECTION_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[13]:
            yield index, index+1, "CONNECTION"
            
def NUMBER_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[14]:
            yield index, index+1, "NUMBER"

def ORDER_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[15]:
            yield index, index+1, "ORDER"
            
def OFFER_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[16]:
            yield index, index+1, "OFFER"
            
def PLANNING_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[17]:
            yield index, index+1, "PLANNING"
            
def SEGMENT_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[18]:
            yield index, index+1, "SEGMENT"

def OTHER_entities_detector(doc):
    for index, tok in enumerate(nlp(doc.text)):
        if tok.text.lower() in updated_labels[19]:
            yield index, index+1, "OTHER"


In [15]:
DIEPROFUN_detector = skweak.heuristics.FunctionAnnotator("DIEPROFUN", DIEPROFUN_entities_detector)
NETWORK_detector = skweak.heuristics.FunctionAnnotator("NETWORK", NETWORK_entities_detector)
HARDWARE_detector = skweak.heuristics.FunctionAnnotator("HARDWARE", HARDWARE_entities_detector)
TIME_detector = skweak.heuristics.FunctionAnnotator("TIME", TIME_entities_detector)
COMPETITOR_detector = skweak.heuristics.FunctionAnnotator("COMPETITOR", COMPETITOR_entities_detector)
PROBLEM_detector = skweak.heuristics.FunctionAnnotator("PROBLEM", PROBLEM_entities_detector)
PERSON_detector = skweak.heuristics.FunctionAnnotator("PERSON", PERSON_entities_detector)
BILLING_detector = skweak.heuristics.FunctionAnnotator("BILLING", BILLING_entities_detector)
COMMUNICATION_detector = skweak.heuristics.FunctionAnnotator("COMMUNICATION", COMMUNICATION_entities_detector)
CONTRACT_detector = skweak.heuristics.FunctionAnnotator("CONTRACT", CONTRACT_entities_detector)
LOCATION_detector = skweak.heuristics.FunctionAnnotator("LOCATION", LOCATION_entities_detector)
MECHANIC_detector = skweak.heuristics.FunctionAnnotator("MECHANIC", MECHANIC_entities_detector)
PROCESS_detector = skweak.heuristics.FunctionAnnotator("PROCESS", PROCESS_entities_detector)
CONNECTION_detector = skweak.heuristics.FunctionAnnotator("CONNECTION", CONNECTION_entities_detector)
NUMBER_detector = skweak.heuristics.FunctionAnnotator("NUMBER", NUMBER_entities_detector)
ORDER_detector = skweak.heuristics.FunctionAnnotator("ORDER", ORDER_entities_detector)
OFFER_detector = skweak.heuristics.FunctionAnnotator("OFFER", OFFER_entities_detector)
PLANNING_detector = skweak.heuristics.FunctionAnnotator("PLANNING", PLANNING_entities_detector)
SEGMENT_detector = skweak.heuristics.FunctionAnnotator("SEGMENT", SEGMENT_entities_detector)
OTHER_detector = skweak.heuristics.FunctionAnnotator("OTHER", OTHER_entities_detector)



In [16]:
annotators_list = [DIEPROFUN_detector, NETWORK_detector, HARDWARE_detector, TIME_detector, 
                  COMPETITOR_detector, PROBLEM_detector, PERSON_detector, 
                   BILLING_detector, COMMUNICATION_detector, CONTRACT_detector, LOCATION_detector,
                  MECHANIC_detector, PROCESS_detector,CONNECTION_detector,NUMBER_detector,
                  ORDER_detector, OFFER_detector, PLANNING_detector, SEGMENT_detector, OTHER_detector]

In [17]:
labels = ["DIEPROFUN", "NETWORK", "HARDWARE",  "TIME", "COMPETITOR", "PROBLEM", "PERSON", "BILLING", "COMMUNICATION","CONTRACT","LOCATION",
         "MECHANIC", "PROCESS", "CONNECTION", "NUMBER", "ORDER", "OFFER", "PLANNING", "SEGMENT", "OTHER"]

In [18]:
doc = nlp("en nu is mijn vraag of waar eventueel mijn firewall en tussen virus en cisco ook naar de tv kunnen kijken of zo, want daar hebben geen ziggo tv abonnement voor, ga voor alleen abonnementen als het goed heb begrepen.")
DIEPROFUN_detector(doc)
HARDWARE_detector(doc)
PROBLEM_detector(doc)
PERSON_detector(doc)
COMPETITOR_detector(doc)


skweak.utils.display_entities(doc, "DIEPROFUN")
print("******NEXT ANNOTATOR******")
skweak.utils.display_entities(doc, "PROBLEM")
print("******NEXT ANNOTATOR******")
skweak.utils.display_entities(doc, "HARDWARE")
print("******NEXT ANNOTATOR******")
skweak.utils.display_entities(doc, "PERSON")
print("******NEXT ANNOTATOR******")
skweak.utils.display_entities(doc, "COMPETITOR")


******NEXT ANNOTATOR******


******NEXT ANNOTATOR******


******NEXT ANNOTATOR******
******NEXT ANNOTATOR******


******NEXT ANNOTATOR******


In [19]:
sample_df = df
sample_df.tail()

,my_problem_is
7336,[customer][nvt][my_problem_is][01:18]: Zo bezi...
7337,[customer][nvt][my_problem_is][00:12]: Ik heb ...
7338,"[customer][nvt][my_problem_is][00:11]: Nee, ni..."
7339,[customer][nvt][my_problem_is][00:09]: Ja hoor...
7340,[customer][unpleasant][my_problem_is][11:09]: ...


In [20]:
texts = sample_df.my_problem_is.values

cleaned_texts = [re.sub("[\(\[].*?[\)\]]", "", conversation) for conversation in texts]
cleaned_texts = [''.join(conversation.lstrip().splitlines()) for conversation in cleaned_texts]
cleaned_texts = [''.join(conversation) for conversation in cleaned_texts if len(conversation.split(" ")) < 550]

In [92]:
print(cleaned_texts[0])
exam = "Ik heb een duo sim en een telefoon allebei van kpn en nu krijg ik weer geregeld voor werk en ons volgende week ook het geval was"
doc = nlp(exam)
print(cleaned_texts[1])

: Een ja, nou, ik heb een abonnement verlengt en een toestel bij het toestel nou, bijzonder laden. ... alleen, dat is een oud adres en daar kom ik niet veranderen omdat ze zeiden dat ik dat mijnkpn moet doen. : Die staat een paar weken geleden contact gehad dat dat dat werd, stond er en dat is toen aangepast, maar nu wil : Die daar proberen, die afleveren, omruilen ben ik natuurlijk niet. : En ik had gisteren proberen te bellen. : Zit alleen even te kijken, want hier vandaan, vanuit kpn gezien kan ik hem nu niet aanpassen omdat die eigenlijk al ja onderweg is om het zo te zeggen aan ja : Echt met postnl. Ja, ik heb een postnl zo proberen aanpassen dat zo n opnemen. Bij een postnl punt weet ik niet of dat toen omdat het een teken : Nee, alleen aanpassen naar of bij de buren of bij een postnl punt. Maar ik kon niet. : Ja, ja, ik weet niet of dat werkt. Dan ondertussen moest ik het pakket is maar : De in dat er gegevens gewijzigd zijn, dus staat niet specifiek zijn bij welke gegevens. Dat

In [27]:
rule_based_annotator = CombinedAnnotator()

for annotator in annotators_list:
    rule_based_annotator.add_annotator(annotator)
    print(annotator)

In [28]:
print("before: ", len(cleaned_texts))
new_texts = cleaned_texts
print("after : ", len(new_texts))


before:  6811
after :  6811


In [29]:
new_docs = []
conll_docs = ""
cleaned_texts = new_texts
for doc in tqdm(nlp.pipe(cleaned_texts, batch_size=50, n_process=-1), total=len(cleaned_texts)):
    doc = rule_based_annotator(doc)
    new_docs.append(doc)

100%|███████████████████████████████████████████████████████████████████████████████| 6811/6811 [29:06<00:00,  3.90it/s]


In [30]:
skweak.utils.docbin_writer(new_docs,"data/annotated_data")

Write to data/annotated_data_3_7...done


In [31]:
hmm = skweak.generative.HMM("hmm", labels=labels)

In [32]:
hmm.fit(new_docs)

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Finished E-step with 6778 documents
Starting iteration 2


         1     -718472.5425             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Finished E-step with 6778 documents
Starting iteration 3


         2     -704921.8529      +13550.6896


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Finished E-step with 6778 documents
Starting iteration 4


         3     -699784.5878       +5137.2651


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Finished E-step with 6778 documents


         4     -697077.8065       +2706.7814


In [41]:
docs = list(hmm.pipe(new_docs))

In [42]:
for doc in tqdm(docs):
    doc.ents = doc.spans["hmm"]

100%|█████████████████████████████████████████████████████████████████████████████| 6811/6811 [00:01<00:00, 6093.34it/s]


In [43]:
new_extracted = docs

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
docs = new_extracted

In [46]:
train, test = train_test_split(docs, test_size=0.30, shuffle=True)

In [47]:
len(train)

4767

In [48]:
len(test)

2044

In [95]:
doc = nlp("Ik heb een duo sim en een telefoon allebei van kpn en nu krijg ik weer geregeld voor werk en ons volgende week ook het geval was")
hmm.pipe(doc)
skweak.utils.display_entities(doc, "hmm")

RuntimeError: Annotation source "hmm" cannot be found

In [50]:
skweak.utils.docbin_writer(train, "data/train_ner_3.spacy")
skweak.utils.docbin_writer(test, "data/test_ner_3.spacy")

Write to data/train_ner_3_7.spacy...done
Write to data/test_ner_3_7.spacy...done


In [52]:
labels_dict = dict.fromkeys(labels, 0)

for doc in test:
  for ent in doc.ents:
    if ent.label_ in labels_dict:
      labels_dict[ent.label_] +=1

print(labels_dict)

{'DIEPROFUN': 5513, 'NETWORK': 1169, 'HARDWARE': 1342, 'TIME': 553, 'COMPETITOR': 0, 'PROBLEM': 3, 'PERSON': 21492, 'BILLING': 2, 'COMMUNICATION': 2, 'CONTRACT': 278, 'LOCATION': 0, 'MECHANIC': 1, 'PROCESS': 0, 'CONNECTION': 47, 'NUMBER': 74, 'ORDER': 0, 'OFFER': 0, 'PLANNING': 0, 'SEGMENT': 100, 'OTHER': 0}


In [55]:
docs = [nlp(text) for text in tqdm(train)]

100%|███████████████████████████████████████████████████████████████████████████████| 2100/2100 [00:24<00:00, 87.22it/s]


In [53]:
class Spacy2Conll:
    def __init__(self):
        pass
    
    def convert(self, docs):
        print("EXTENDED DOCS {}".format(len(docs)))
        self.result = list()
        docstart = "-DOCSTART-"
        empty = "O"
        for index, doc in enumerate(tqdm(docs)):

            self.result.append(docstart)
            self.result.append("\n")
            self.result.append("\n")
            for t_index,tok in enumerate(doc):
                    label = tok.ent_iob_
                    if tok.ent_iob_ != "O":
                        label += '-' + tok.ent_type_
                    if not str(tok).isspace():

                        self.result.append(str(t_index+1)  +"\t" + str(tok) + "\t" + label)
                        self.result.append("\n")
                    if str(tok) in [".", "?"]:
                        self.result.append("\n")

            self.result.append("\n")

    def write(self, output_path: str):

        with open(output_path, "w",encoding="utf-8") as outfile:
            outfile.write("".join(self.result))
        print("CoNLL done")


In [54]:
converter = Spacy2Conll()

In [56]:
converter.convert(test)
converter.write("data/test_4_14_.txt")

EXTENDED DOCS 2044


100%|█████████████████████████████████████████████████████████████████████████████| 2044/2044 [00:00<00:00, 2621.79it/s]

CoNLL done


In [120]:
def append_list(sim_words, words):
    
    list_of_words = []
    
    for i in range(len(sim_words)):
        
        sim_words_list = list(sim_words[i])
        sim_words_list.append(words)
        sim_words_tuple = tuple(sim_words_list)
        list_of_words.append(sim_words_tuple)
        
    return list_of_words

input_word = 'buurt,draadloos,glasvezel,monteur,databundel'
user_input = [x.strip() for x in input_word.split(',')]
result_word = []
    
for words in user_input:
    
        sim_words = similarity_model.wv.most_similar(words, topn = 5)
        sim_words = append_list(sim_words, words)
            
        result_word.extend(sim_words)
    
similar_word = [word[0] for word in result_word]
similarity = [word[1] for word in result_word] 
similar_word.extend(user_input)
labels = [word[2] for word in result_word]
label_dict = dict([(y,x+1) for x,y in enumerate(set(labels))])
color_map = [label_dict[x] for x in labels]

In [121]:
import plotly.io as pio
pio.renderers.default = 'iframe'
import numpy as np
import plotly.graph_objs as go
from sklearn.decomposition import PCA

def display_pca_scatterplot_3D(model, user_input=None, words=None, label=None, color_map=None, topn=5, sample=2):

    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]
    
    word_vectors = np.array([model.wv[w] for w in words])
    

    two_dim = PCA(random_state=0).fit_transform(word_vectors)[:,:2]

    data = []
    count = 0
    
    for i in range (len(user_input)):

                trace = go.Scatter(
                    x = two_dim[count:count+topn,0], 
                    y = two_dim[count:count+topn,1],  
                    # z = three_dim[count:count+topn,2],
                    text = words[count:count+topn],
                    name = user_input[i],
                    textposition = "top center",
                    textfont_size = 20,
                    mode = 'markers+text',
                    marker = {
                        'size': 10,
                        'opacity': 0.8,
                        'color': 2
                    }
       
                )
                
                # For 2D, instead of using go.Scatter3d, we need to use go.Scatter and delete the z variable. Also, instead of using
                # variable three_dim, use the variable that we have declared earlier (e.g two_dim)
            
                data.append(trace)
                count = count+topn

    trace_input = go.Scatter(
                    x = two_dim[count:,0], 
                    y = two_dim[count:,1],  
                    # z = three_dim[count:,2],
                    text = words[count:],
                    name = 'input words',
                    textposition = "top center",
                    textfont_size = 20,
                    mode = 'markers+text',
                    marker = {
                        'size': 10,
                        'opacity': 1,
                        'color': 'black'
                    }
                    )

    # For 2D, instead of using go.Scatter3d, we need to use go.Scatter and delete the z variable.  Also, instead of using
    # variable three_dim, use the variable that we have declared earlier (e.g two_dim)
            
    data.append(trace_input)
    
# Configure the layout

    layout = go.Layout(
        margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
        showlegend=True,
        legend=dict(
        x=1,
        y=0.5,
        font=dict(
            family="Courier New",
            size=25,
            color="black"
        )),
        font = dict(
            family = " Courier New ",
            size = 15),
        autosize = False,
        width = 1000,
        height = 1000
        )


    plot_figure = go.Figure(data = data, layout = layout)
    plot_figure.show()

    
display_pca_scatterplot_3D(similarity_model, user_input, similar_word, labels, color_map)

In [82]:
def export_training_data(spacy_docs: List, split: str, mode="BIO"): #NEW VERSION WITH SENTENCES
        
    document_registry = list()
    annotated_spacy_docs = spacy_docs
    document = list()
    
    if mode == "DOCRED":
        print(f'Output type: {mode}')
    if mode == "conllu":
        print(f'Output type: {mode}')
        document.append("# global.columns = "+("\t").join(["id","text","ner"]))
        # document.append("\n")
    for doc in annotated_spacy_docs:
        
        word_positions_dict = dict()
        current_document = []
    
        tags=[token.ent_iob_+"-"+"_".join((token.ent_type_.split(" "))) if token.ent_iob_ != "O" else "O" for token in doc]
        
        biluo_tags = iob_to_biluo(tags)
        
        for i,tag in enumerate(biluo_tags):
            if tag.startswith("U-"):
                biluo_tags[i] = tag.replace("U-","S-")
            if tag.startswith("L-"):
                biluo_tags[i] = tag.replace("L-","E-")
                
        tags = biluo_tags
                
        document.append("-DOCSTART-")
        document.append("\n\n")
        

        for i,word_tag in enumerate(zip(doc,tags)):
            document.append(("\t").join([str(word_tag[0]),str(word_tag[1])]))
            if str(word_tag[0]) == "." or str(word_tag[0]) == "?":
                document.append("\n")
            document.append("\n")
                
        
    output = f'data/randomSplits/{split}.txt'
    with open(output, "w",encoding="utf-8") as outfile:
        outfile.write("".join(document))

In [83]:
export_training_data(train, "train","BIO")

In [85]:
sample_size = [100,200,500,1000,2000]

for size in sample_size:
    selection = np.random.choice(train, size, replace=False)
    export_training_data(selection,f'train_{size}',"BIO")

/tmp/ipykernel_1638/2067422047.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  selection = np.random.choice(train, size, replace=False)
